# Imports, Functions & Testing

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import pickle
import reload
from bs4 import BeautifulSoup

In [2]:
def wiki_senate_scraper():
    import pandas as pd
    import numpy as np
    import requests
    from bs4 import BeautifulSoup
    
    ## Starting page + Q.C. of response
    start_url = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
    start_resp = requests.get(start_url)
    print(f'Starting Response: {start_resp}')
    
    ## Creating soup + pulling all links for senate elections after 17th amendment
    start_soup = BeautifulSoup(start_resp.text, 'html.parser')
    start_links = start_soup.findAll('a')
    start_sen_links = start_links[78:134] ## Previously located
    
    ## Base of url for all senate election pages
    base_url = 'https://en.wikipedia.org'
    
    ## Loop same process for all links + storage
    yr_dfs = {}
    yr_tocs = {}
    count = 0
    for link in start_sen_links:

        ## Q.C during execute
        count += 1
        if count in [25, 50]:
            print('Checkpoint! (25 loops)')
        
        ## Collecting strings for use
        end_url = link.get('href')
        year = link.get_text()
        full_url = base_url + end_url
        
        ## Making soup + collecting all tables
        link_resp = requests.get(full_url)
        link_soup = BeautifulSoup(link_resp.text, 'html.parser')
        link_tables = link_soup.findAll('table', attrs={'class': ['wikitable', 'infobox vevent', 'infobox']})
        
        ## Collecting list of states with elections in each year
        link_toc = link_soup.find('div', attrs={'id':'toc'})
        link_toc = link_toc.findAll('a', href=is_state)
        toc_list = [tag.get('href').replace('#', '') for tag in link_toc]
        
        ## Converting to dataframe + storage
        elect_df = pd.read_html(str(link_tables))
        yr_dfs[year] = elect_df
        yr_tocs[year] = toc_list
    
    print(f'Total pages scraped: {count}')
    return yr_dfs, yr_tocs

## https://www.crummy.com/software/BeautifulSoup/bs4/doc/#kinds-of-filters

In [3]:
def is_state(href):
    states_list = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
                   'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
                   'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
                   'Kentucky', 'Louisiana', 'Maine', 'Maryland',
                   'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                   'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New_Hampshire',
                   'New_Jersey', 'New_Mexico', 'New_York', 'North_Carolina',
                   'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                   'Rhode_Island', 'South_Carolina', 'South_Dakota',
                   'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
                   'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming']
    for state in states_list:
        if state in href:
            return href

In [4]:
def election_collector(dict_tables, dict_lists, yr_end='2020'):
    
    ## Requirement of proper dictionaries
    if dict_tables.keys() != dict_lists.keys():
        print('**WARNING**')
        print('Keys do not match in dictionaries passed. Adjust and try again.')
        return '***'*10
    
    ## Containers for results
    coll_elects = {}
    
    ## List creation for looping through dicts
    yr_list = list(dict_lists.keys())
    
    ## Looping + storage
    for year in yr_list:
        if year == yr_end:
            break
        print('***'*10)
        print(f"Collecting {year}'s elections...")
        yr_tables = dict_tables[year]
        yr_toc = dict_lists[year]
        
        ## Creating containers to further separate data
        yr_sum_ldrs = yr_tables[2]
        most_tables = yr_tables[3:]
        yr_summary = []
        yr_states = []
        count = 0
        
        for i, df in enumerate(most_tables):
            if count < 1:
                if year == '2018':
                    count += 1
                    yr_summary.append(yr_tables[10])
                    yr_summary.append(yr_tables[11])
                    continue
                elif df.shape[1] is 6 and most_tables[i+1].shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
                    yr_summary.append(most_tables[i+1])
                    continue
                elif df.shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
            
            if df.shape[1] in [4,5,6] and df.shape[0] <= 15:
                yr_states.append(most_tables[i])
        
        coll_elects[year] = [yr_sum_ldrs, yr_summary, yr_states]
        
    print('\n')
    print('---'*20)
    disp_yrs = list(coll_elects.keys())
    print(f'Collection Complete: Data is for {disp_yrs[0]} - {disp_yrs[-1]}')
    print('---'*20)
    return coll_elects

In [5]:
def yr_summary_collector(election_collection):
    
    ## Container for results
    yr_summaries = {}
    
    ## Extraction as a list for iteration
    for year in election_collection:
        elect_list = [election_collection[year]]
        
        ## Looping through year's collection
        for coll in elect_list:
            ## Container for mid-results
            holder = []
            ## Looping through each table in 'year summary' group
            for table in coll[1]:
                ## Variable to filter unwanted tables out
                has_elected = False
                ## Looping through each row in 'results' column (idx 4)
                for u in table.iloc[:,4]:
                ## Checking for str to 'flag' table as collectable
                    if isinstance(u, str) and 'elected' in u:
                        has_elected = True
                if has_elected == True:
                    holder.append(table)
            ## Check if multiple tables in 'year summary' group
            if len(holder) > 1:
                is_match = holder[0].iloc[1,:] == holder[1].iloc[1,:]
                ## Check if tables have matching headers for special 
                ## election years + storage
                if is_match.sum() in [2,3]:
                    fin_tab = holder[1].append(holder[0], ignore_index=True)
                    yr_summaries[year] = fin_tab
            ## Storage of non-special election years
            else:
                try:
                    yr_summaries[year] = holder.pop()
                except:
                    continue
        
    return yr_summaries

In [6]:
def ref_tabler(summary_df, mod_df=True):
    ## Containers for mid-results
    st_list = []
    sntr_list = []
    
    ## Looping through states' names
    for state in summary_df['State']:
        ## Check/replace spaces with '_' + storage
        if ' ' in state:
            fmt_state = state.replace(' ', '_')
            st_list.append(fmt_state)
        else:
            st_list.append(state)
    ## Looping + storing senator names
    for name in summary_df['Senator']:
        sntr_list.append(name)

    ## Dataframe with Sentors as index + states as values
    state_ref_df = pd.DataFrame({'State_id':st_list, 'Incumbent':sntr_list})
    #state_ref_df.columns = ['State_id']
    
    ## Modifying list of states to that of the ref table OR not + return
    if mod_df:
        summary_df['State'] = st_list
        return summary_df, state_ref_df
    else:
        return state_ref_df

In [7]:
def master_tabler(yr_summary_dict):

    holder = []

    for year in yr_summary_dict:
        holder.append(yr_summary_dict[year][1])

    name_lookup_df = pd.concat(holder, ignore_index=True)
    
    return name_lookup_df

In [8]:
def yr_sum_formatter(yr_sum_dict, ref_table=True):
    ## Container for results + column helper
    res_dict = {}
    df_cols = ['State', 'Senator', 'Party', 'Electoral_History', 'Results', 'Candidates']
    
    ## Looping to each year's summary table
    for year in yr_sum_dict:
        ## Setting copy to modify + setting columns + rows to remove
        summary_df = yr_sum_dict[year].copy()
        summary_df.columns = df_cols
        drop_1 = summary_df.iloc[0]
        drop_2 = summary_df.iloc[1]
        
        ## Removing rows that match 'drop_' rows (former column info)
        for idx, row in summary_df.iterrows():
            if drop_1.equals(row):
                summary_df.drop(idx, inplace=True)
            elif drop_2.equals(row):
                summary_df.drop(idx, inplace=True)
        
        ## Resetting index to rangeIndex
        summary_df.reset_index(drop=True, inplace=True)
        
        ## Storing State-Senator lookup dataframe OR not
        if ref_table:
            summary_df, state_ref_df = ref_tabler(summary_df)
            state_ref_df['Year'] = year
            res_dict[year] = [summary_df, state_ref_df]    
        else:
            res_dict[year] = summary_df
        
    ## Return results
    return res_dict

In [9]:
def st_election_cleaner(dict_of_lists, exclude=[]):
    st_gens_cln = {}

    for year in dict_of_lists:
        if year in exclude:
            continue
        coll = dict_of_lists[year]
        holder = []
        
        for table in coll:
            if table.shape[1] in [5,6]:
                try:
                    table_cln = st_election_formatter(table)
                    holder.append(table_cln)
                except KeyError as e:
                    print(f'Error!! {e}')
                    print(f'Year affected: {year}')
                    display(table)
        
        st_gens_cln[year] = holder
        
    return st_gens_cln

In [10]:
def st_election_collector(election_collection):
    
    ## Container for results
    st_elects = {}
    
    ## Extraction of each year's state elections
    for year in election_collection:
        elect_list = election_collection[year]
        st_elects[year] = elect_list[2]
        
    return st_elects

In [11]:
def st_election_formatter(df):
    df_cln = df.copy()
    
    cols = ['Drop']
    cols.extend(df_cln.iloc[0,:-1])
    df_cln.columns = cols
    df_cln.drop(0, inplace=True)
    
    for i, v, na in zip(df_cln.index, df_cln['Drop'], df_cln['Drop'].isna()):
        if not na:
            df_cln[v] = df_cln['Party'][i]
            
    df_cln.drop(columns='Drop', inplace=True)
    
    for col in df_cln:
        df_cln_height = df_cln.shape[0]
        df_cln_nNA = df_cln[col].isna().sum()
        
        if df_cln_nNA == df_cln_height:
            df_cln.drop(columns=col, inplace=True)
            
        elif df_cln_height == 2 and df_cln_nNA > 0:
            df_cln.dropna(axis=0, inplace=True)
            
        elif df_cln_nNA == (df_cln_height-1) and df_cln_nNA > 0:
            df_cln.drop(columns=col, inplace=True)
            
        elif df_cln_nNA > 0:
            for i, v, na in zip(df_cln.index, df_cln[col], df_cln[col].isna()):
                if na:
                    df_cln.drop(i, inplace=True)
    
    return df_cln

In [12]:
def table_checker(list_=None, dict_=None):
    if not list_ and not dict_:
        print('---'*20)
        print('Error! Must enter either List or Dictionary.')
        print('---'*20)
        
    if list_ and not dict_:
        slice_ = input('Index(ices) to slice:')
        try:
            res = list_[slice_]
            return res
        except:
            print(f'Error! List slice {slice_}.')
            
    if dict_ and not list_:
        key_ = input('Key to pull:')
        try:
            res = dict_[key_]
            return res
        except:
            print(f'Error! Dictionary key {key_}')

# Initial Data

> [Link to Dataset](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PEJ5QU)

In [ ]:
df = pd.read_csv('dataverse_files/1976-2018-senate.csv', encoding='latin-1')
df.head()

In [ ]:
df.info()

In [ ]:
df[(df['state_po'] == 'AZ') & (df['year'] == 1976)]

In [ ]:
# len(df[df['candidate'].isna()])

# df['writein'].value_counts()

# for col in df:
#     display(df[col].value_counts())

# pd.Series.value_counts()

# Additional Data

Current set of features includes:
* Year of election
* State (encoded)
* 

> [Link to Brookings](https://www.brookings.edu/multi-chapter-report/vital-statistics-on-congress/)

> [Link to BallotPedia](https://ballotpedia.org/Legislative_Branch)

> [Link to WikiPedia](https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections)

> [Link to Wiki-Category (U.S. Senator)](https://commons.wikimedia.org/wiki/Category:Senators_of_the_United_States)

## Testing tables 'infobox vevent'

In [ ]:
# url3 = 'https://en.wikipedia.org/wiki/1978_United_States_Senate_elections'
# response3 = requests.get(url3)
# print(response3)

# soup3 = BeautifulSoup(response3.text, 'html.parser')
# wiki_tables3 = soup3.findAll('table', attrs={'class': ['wikitable', 'infobox vevent']})

# print(type(wiki_tables3))

# test_df3 = pd.read_html(str(wiki_tables3))

# tester = test_df3[0]

In [ ]:
# https://stackoverflow.com/questions/27156278/index-pandas-dataframe-by-column-numbers-when-column-names-are-integers
# tester.columns = ['a', 'b', 'c']
# display(tester.iloc[6:18,0:3])

# display(test_df3[2])

In [ ]:
# url = 'https://en.wikipedia.org/wiki/1976_United_States_Senate_elections'
# response = requests.get(url)
# print(response)

# soup = BeautifulSoup(response.text, 'html.parser')
# wiki_tables = soup.findAll('table', attrs={'class': 'wikitable'})

# type(wiki_tables)

In [ ]:
# test_df = pd.read_html(str(wiki_tables))
# display(test_df[0])

# len(test_df)

## Pulling state election tables

In [ ]:
# ## Pulling Arizona gen election results
# tester = test_df[5].copy()
# print('O.G. Copy:')
# print('---'*20)
# display(tester)

# ## Setting columns manually + Q.C.
# tester.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
# print('\nNew Column Headers:')
# print('---'*20)
# display(tester)

# ## Dropping irrelevant information + Q.C.
# tester.drop(index=[0, 6, 8], inplace=True)
# print('\nDropping Rows:')
# print('---'*20)
# display(tester)

# ## Flip and place Turnout as its own feature + Q.C.
# for idx, v, k in zip(tester.index, tester.Null, tester.Null.isna()):
#     if not k:
#         #print(idx, v)
#         tester[v] = tester.Party[idx]

# ## Removing the row with turnout info/column with nulls + Q.C.
# try:
#     tester.drop(columns='Null', index=7, inplace=True)
#     print('\nFinal Version:')
#     print('---'*20)
#     display(tester)
# except:
#     print('Something went wrong!')

In [ ]:
# type(tester['Votes'][5]) # str

In [ ]:
# tester2 = gen_election_cleaner(test_df[6], verbose=2)

In [ ]:
# test_df[-1]

In [ ]:
# url2 = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [ ]:
# url2 = 'https://en.wikipedia.org/wiki/2018_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [ ]:
# soup2 = BeautifulSoup(response2.text, 'html.parser')
# wiki_toc = soup2.find('div', attrs={'id':'toc'})
# wiki_toc = wiki_toc.findAll('a', href=is_state)
# display(wiki_toc[0:5])
# jiji = wiki_toc[0:5]

# jiji = [tag.get('href').replace('#', '') for tag in jiji]
# jiji = jiji
# jiji

## Working

In [ ]:
# test = wiki_links[78:134]
# beeb = test[0].get_text()
# geeb = 'https://en.wikipedia.org'
# geeb + beeb

In [ ]:
# display(elect_yr_dfs.keys())
# display(elect_yr_tocs.keys())
# display(len(elect_yr_dfs) == len(elect_yr_tocs))
# elect_yr_dfs.keys() != elect_yr_tocs.keys()
# list(elect_yr_dfs.keys())

In [ ]:
# elect_yr_dfs['1918']

In [ ]:
# elect_yr_tocs['1918']

In [ ]:
# year = '1924'

# for i, df in enumerate(elect_yr_dfs[year]):
#     print(i, df.shape)

# display(elect_yr_dfs[year][10])
# elect_yr_dfs[year][10].shape[1] is 6

In [ ]:
# koko = {v:elect_yr_dfs[v] for v in ['1918', '1924', '1976', '2016', '2018']}
# lplp = {v:elect_yr_tocs[v] for v in ['1918', '1924', '1976', '2016', '2018']}

In [ ]:
# year = '2016'

# for v in ftft[year]:
#     print(len(v))
    
# ftft[year][1]

In [ ]:
# yuyu = test_res[-1].append(test_res[-2], ignore_index=True)
# yuyu

In [ ]:
# yuyu = test_res[-1].iloc[1,:] == test_res[-2].iloc[1,:]
# yuyu.sum()

In [ ]:
# len(test_res)

## Getting it together

* In 2016, year summary is index 10 no shape conflict
* In 2018, year summary tables are index 10, 11
* In 2020, year summary tables are index 10, 11 but no shape conflict
* In 2022, year summary **is not worth it**

In [ ]:
# for i, table in enumerate(elect_yr_tables['2018']):
#     print(f'Table {i}:')
#     display(table)
#     if i == 15:
#         break

In [ ]:
# for u in election_collection_raw[year][1][0].iloc[:,4]:
#     if isinstance(u, str) and 'elected' in u:
#         print('faslidfjasedfhaedfng')
        
# for u in election_collection_raw[year][1][1].iloc[:,4]:
#     if isinstance(u, str) and 'elected' in u:
#         print('faslidfjasedfhaedfng')

In [ ]:
# print(election_collection_raw[year][1][0].shape)
# print(election_collection_raw[year][1][1].shape)

In [ ]:
# test_list = [election_collection_raw[year][2][8], election_collection_raw[year][2][3]]
# for u in test_list:
    
#     display(u.iloc[:,2])

In [ ]:
# elect_yr_tables, elect_yr_tocs = wiki_senate_scraper()

# election_collection_raw = election_collector(elect_yr_tables, elect_yr_tocs)

In [ ]:
# election_collection_raw[year][1][1]
# for i, df in enumerate(election_collection_raw[year][2]):
#     display(df)
# elect_yr_tables['2006'][3:]

## Loading `Election_Collection`

In [ ]:
# with open('election_collection_raw.pickle', 'wb') as f:
#     pickle.dump(election_collection_raw, f)
#     f.close()

with open('election_collection_raw.pickle', 'rb') as f:
    election_collection_raw = pickle.load(f)
    f.close()

## Must do this befor running code for state elections. Error in code
do_not_use = election_collection_raw['2018'][2].pop(0)

## Separation/Cleaning

### Testing

In [ ]:
# test_res_dict = yr_sum_formatter(test_res)
# for table in test_res_dict['2008']:
#     display(table)

In [ ]:
# year = '2006'

# display(election_collection_raw.keys())
# print('\n')
# for grp in election_collection_raw[year]:
#     print(len(grp))
# print('\n')
# display(election_collection_raw[year][0])
# display(election_collection_raw[year][1])
# display(election_collection_raw[year][2])

In [ ]:
# test_list = []
# test_list.append(election_collection_raw['1918'])
# test_list.append(election_collection_raw['1920'])
# test_list.append(election_collection_raw['1976'])
# test_list.append(election_collection_raw['2006'])
# test_list.append(election_collection_raw['2008'])

# test_list = ['1918', '1920', '1976', '2006', '2008']
# test_dict = {}
# for year in test_list:
#     test_dict[year] = election_collection_raw[year]
# test_dict['2008']

In [ ]:
# type(election_collection_raw['1994'])

In [ ]:
# print(type(election_collection_raw['1920']))
# print(type(election_collection_raw['1976']))
# type(test_res['1920'])

In [ ]:
# test_res = yr_summary_collector(test_dict)
# for t in test_res:
#     display(test_res[t])

In [ ]:
# def gen_election_cleaner(df, verbose=1):
#     import pandas as pd
    
#     df_cln = df.copy()
    
#     if verbose in [1,2]:
#         ## Printing startpoint
#         print('O.G. Copy:')
#         print('---'*20)
#         display(df_cln)

#     ## Setting columns manually + Q.C.
#     df_cln.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
#     if verbose == 2:
#         print('\nNew Column Headers:')
#         print('---'*20)
#         display(df_cln)

#     ## Dropping irrelevant information + Q.C.
#     df_cln.drop(index=[0, 6, 8], inplace=True)
#     if verbose == 2:
#         print('\nDropping Rows:')
#         print('---'*20)
#         display(df_cln)

#     ## Flip and place Turnout as its own feature + Q.C.
#     for idx, v, k in zip(df_cln.index, df_cln['Null'], df_cln['Null'].isna()):
#         if not k:
#             df_cln[v] = df_cln['Party'][idx]

#     ## Removing the row with turnout info/column with nulls + Q.C.
#     df_cln.drop(columns='Null', index=7, inplace=True)
#     if verbose in [1,2]:
#         print('\nFinal Version:')
#         print('---'*20)
#         display(df_cln)

#     return df_cln

### Working..

**To map the tables to states I need to:**
* Loop through `election_collection_raw`
* Grab each section of state summaries
* Loop through each section
* Clean each state summary
* Place into container
* Loop through container
* Map each senator to the state
* Update table

In [ ]:
# year = '1994'

# for i, table in enumerate(election_collection_raw[year][2]):
#     print(i)
#     display(table)

# ## Confirmed idx 7, 9, 14 yr 1994
# test_5 = election_collection_raw[year][2][4]
# test_5

# # pd.DataFrame.drop()
# print(f'DF shape: {test_5.shape}')
# st_election_formatter(test_5)

# yr_st_dict['1994']
# yr_st_elects['2018'][0]

# display(yr_sum_dict['2020'])

In [15]:
## This works for all years up until 2018 (doesnt have these tables)
yr_sum_tables = yr_summary_collector(election_collection_raw)
yr_sum_dict = yr_sum_formatter(yr_sum_tables)
name_lookup_df = master_tabler(yr_sum_dict)

# display(name_lookup_df['Year'].unique())
print(len(name_lookup_df['Year'].unique()))
print(name_lookup_df.shape) #1899, 3
# name_lookup_df.head(10)

yr_st_elects = st_election_collector(election_collection_raw)
print(len(yr_st_elects))
yr_st_dict = st_election_cleaner(yr_st_elects)

In [24]:
def st_election_aggregator(dict_of_lists):
    
    ## Container for results
    res_dict = {}
    
    ## Looping thorough dict, setting each key as list to iterate over
    for year in dict_of_lists:
        year_list = dict_of_lists[year]
        holder = []
        
        ## Looping through each year pulling state generals
        for table in year_list:
            if len(table['Party'].unique()) > 1:
                display(table)
                holder.append(table)
                
        ## Try/except incase a year doesn't have that info
        ## Concats and stores final df into res_dict
        try:
            yr_df = pd.concat(holder, sort=False, ignore_index=True)
            res_dict[year] = yr_df
        except ValueError as e:
            print(f'{e}! Year {year}')
            res_dict[year] = holder
    
    return res_dict

In [19]:
def st_election_state_mapper(dict_of_dfs, lookup_ref):
    
    new_dict = {}

    ## Loop through keys i.e. years + container for cleaning
    for year in dict_of_dfs:
        #holder = []
        ## Loop through each election table + set col for incumbents
        if isinstance(dict_of_dfs[year], pd.DataFrame):
            table = dict_of_dfs[year]
            
            try:
                table['Incumb_Y'] = 0
                table['State'] = None
                if 'Candidate' in table.columns:
                    ## Iterate over candidate's names 
                    for i, name in enumerate(table['Candidate']):
                        ## Check if incumbent + prep for state lookup
                        if '(Incumbent)' in name:
                            name = name.replace(' (Incumbent)', '')
                            table['Incumb_Y'].iloc[i] = 1

                        ## Helps to determine which are general elections + adding state
                        if name in list(lookup_ref['Incumbent']):
                            state = name_lookup_df[lookup_ref['Incumbent'] == name].copy()
                            state_n = state['State_id'].values

                            ## Accounting for special elections
                            for st in state_n:
                                if not '(' in st:
                                    table['State'][i] = st

            except Exception as e:
                    print(f'Year:{year}, {e}, {e.args}')
                    display(table)
                    keep = input('Error with build, keep table? (Y/N):')
                    if keep is 'Y':
                        new_dict[year] = table
                        print(f'Table {year} saved!')
                    elif keep is 'N':
                        break

            ## Storing each table for that year
            #holder.append(table)

            ## Storing year into dictionary
            new_dict[year] = table
        
    return new_dict

In [20]:
# st_election_state_mapper({'test':test}, name_lookup_df)

In [ ]:
yr_st_dict_joined = st_election_aggregator(yr_st_dict)
print(('---'*10)+'Step 1 Complete!'+('---'*10))
# yr_st_dict_mapped = st_election_state_mapper(yr_st_dict_joined, name_lookup_df)

No objects to concatenate! Year 1914
No objects to concatenate! Year 1916
No objects to concatenate! Year 1918


,Party,Candidate,Votes,%,Majority,Turnout
1,Democratic,Oscar Underwood (Incumbent),154664,66.10%,77327,233985
2,Republican,L. H. Reynolds,77337,33.05%,77327,233985
3,Socialist,A. M. Forsman,1984,0.85%,77327,233985


,Party,Candidate,Votes,%,Majority,Turnout
1,Democratic,James Thomas Heflin,160680,69.27%,92220,231960
2,Republican,C. P. Lunsford,68460,29.51%,92220,231960
3,Socialist,W. H. Chichester,2820,1.22%,92220,231960


,Party,Candidate,Votes,%,Majority,Turnout
1,Republican,Ralph H. Cameron,35893,55.17%,6724,65062
2,Democratic,Marcus A. Smith (Incumbent),29169,44.83%,6724,65062


,Party,Candidate,Votes,%,Majority,Turnout
1,Democratic,Thaddeus H. Caraway,126477,65.92%,61096,191858
2,Republican,Charles F. Cole,65381,34.08%,61096,191858


...

,Party,Candidate,Votes,%,Majority,Turnout
1,Democratic,Daniel Inouye (Incumbent),241887,73.57%,154977,328797
2,Republican,Frank Hutchinson,86910,26.43%,154977,328797


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Republican,Steve Symms (Incumbent),196958,51.56%,+1.81%,11892,382024
2,Democratic,John V. Evans,185066,48.44%,-0.33%,11892,382024


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Democratic,Alan J. Dixon (Incumbent),2033783,65.1%,+9.1%,980049,3122883
2,Republican,Judy Koehler,1053734,33.7%,-8.8%,980049,3122883


,Party,Candidate,Votes,%,Majority,Turnout
1,Republican,Dan Quayle (Incumbent),936143,60.57%,340951,1545563
2,Democratic,Jill L. Long,595192,38.51%,340951,1545563
3,Libertarian,Bradford Warren,8314,0.54%,340951,1545563
4,American,Rockland Snyder,5914,0.38%,340951,1545563


,Party,Candidate,Votes,%,±,Majority,Turnout
1,Republican,Chuck Grassley (Incumbent),588880,66.04%,+12.55%,289474,891762
2,Democratic,John P. Roehrick,299406,33.57%,-11.97%,289474,891762


In [23]:
test = table_checker(dict_=yr_st_dict_mapped)
print(type(test))
test

Key to pull:2006
<class 'pandas.core.frame.DataFrame'>


,Party,Candidate,Votes,%,±,Majority,Turnout,Invalid or blank votes,Total votes,Plurality,Incumb_Y,State
0,Republican,Jon Kyl (Incumbent),814398,53.34%,-25.98%,150257,1526782,NaN,NaN,NaN,1,Arizona
1,Democratic,Jim Pederson,664141,43.50%,+43.50%,150257,1526782,NaN,NaN,NaN,0,None
2,Libertarian,Richard Mack,48231,3.16%,-1.90%,150257,1526782,NaN,NaN,NaN,0,None
3,Republican,Dick Mountjoy,1560472,100.00%,NaN,NaN,NaN,NaN,NaN,NaN,0,None
4,American Independent,Don J. Grundmann,30787,100.00%,NaN,NaN,NaN,NaN,NaN,NaN,0,None
5,Libertarian,Michael S. Metti,16742,100.00%,NaN,NaN,NaN,NaN,NaN,NaN,0,None
6,Peace and Freedom,Marsha Feinland,4109,100.00%,NaN,NaN,NaN,NaN,NaN,NaN,0,None
7,Democratic,Dianne Feinstein (incumbent),5076289,59.43%,NaN,NaN,NaN,357583,8899059,NaN,0,None
8,Republican,Dick Mountjoy,2990822,35.02%,NaN,NaN,NaN,357583,8899059,NaN,0,None
9,Green,Todd Chretien,147074,1.72%,NaN,NaN,NaN,357583,8899059,NaN,0,None


In [ ]:
test = list(name_lookup_df['Incumbent'].head(10))


if 'James Paul Clarke' in test:
    tester = name_lookup_df[name_lookup_df['Incumbent'] == 'James Paul Clarke']['State_id']
    print(type(tester))
tester.values

for i in tester:
    if not '(' in i:
        print(i)

In [18]:
test_years = ['1918', '1952', '1980', '2006', '2018']
test_dict = {}

for y in test_years:
    test_dict[y] = yr_st_dict[y]
    
print(test_dict.keys())

dict_keys(['1918', '1952', '1980', '2006', '2018'])


STARTER CODE FOR MAPPING FUNCTION

In [28]:
table_checker(dict_=test_dict_fin)

Key to pull:2006


,%,Candidate,Incumb_Y,Invalid or blank votes,Majority,Party,Plurality,State,Total votes,Turnout,Votes,±
0,53.34%,Jon Kyl (Incumbent),1,NaN,150257,Republican,NaN,Arizona,NaN,1526782,814398,-25.98%
1,43.50%,Jim Pederson,0,NaN,150257,Democratic,NaN,Arizona,NaN,1526782,664141,+43.50%
2,3.16%,Richard Mack,0,NaN,150257,Libertarian,NaN,Arizona,NaN,1526782,48231,-1.90%
3,100.00%,Dick Mountjoy,0,NaN,NaN,Republican,NaN,NaN,NaN,NaN,1560472,NaN
4,100.00%,Don J. Grundmann,0,NaN,NaN,American Independent,NaN,NaN,NaN,NaN,30787,NaN
5,100.00%,Michael S. Metti,0,NaN,NaN,Libertarian,NaN,NaN,NaN,NaN,16742,NaN
6,100.00%,Marsha Feinland,0,NaN,NaN,Peace and Freedom,NaN,NaN,NaN,NaN,4109,NaN
7,59.43%,Dianne Feinstein (incumbent),0,357583,NaN,Democratic,NaN,NaN,8899059,NaN,5076289,NaN
8,35.02%,Dick Mountjoy,0,357583,NaN,Republican,NaN,NaN,8899059,NaN,2990822,NaN
9,1.72%,Todd Chretien,0,357583,NaN,Green,NaN,NaN,8899059,NaN,147074,NaN


In [21]:
test_dict_fin['1980']

,%,Candidate,Incumb_Y,Majority,Party,State,Total votes,Turnout,Votes,±
0,50.15%,Jeremiah Denton,0,40187,Republican,Alabama,NaN,1296757,650362,NaN
1,47.05%,Jim Folsom,0,40187,Democratic,Alabama,NaN,1296757,610175,NaN
2,1.23%,Michael R. A. Erdey,0,40187,Conservative,Alabama,NaN,1296757,15989,NaN
3,1.01%,William A. Crew,0,40187,Libertarian,Alabama,NaN,1296757,13098,NaN
4,0.23%,Sallie M. Hadnott,0,40187,National Democrat,Alabama,NaN,1296757,2973,NaN
5,0.20%,Jim Partain,0,40187,Statesman Party,Alabama,NaN,1296757,2649,NaN
6,0.12%,Mohammed Oliver,0,40187,Socialist Workers,Alabama,NaN,1296757,1511,NaN
7,53.69%,Frank Murkowski,0,12152,Republican,Alaska,NaN,156762,84159,+11.97%
8,45.93%,Clark Gruening,0,12152,Democratic,Alaska,NaN,156762,72007,-12.35%
9,49.46%,Barry Goldwater (Incumbent),1,9399,Republican,Arizona,NaN,874225,432371,NaN


Check why some tables are not getting matched to states

In [71]:
name_lookup_df[name_lookup_df['State_id'] == 'North_Dakota']

,State_id,Incumbent,Year
21,North_Dakota,Asle Gronna,1914
54,North_Dakota,Porter J. McCumber,1916
132,North_Dakota,Asle J. Gronna,1920
164,North_Dakota,Porter J. McCumber,1922
239,North_Dakota,Gerald P. Nye,1926
275,North_Dakota,Lynn J. Frazier,1928
353,North_Dakota,Gerald P. Nye,1932
387,North_Dakota,Lynn Frazier,1934
465,North_Dakota,Gerald Nye,1938
502,North_Dakota,Lynn Frazier,1940


## Break

In [ ]:
for year in yr_sum_dict:
    coll = yr_sum_dict[year]
    for tab in coll:
        display(tab)

In [ ]:
count = 0
test_list = []
for table in election_collection_raw[year][2]:
    try:
        res = gen_election_cleaner(table, verbose=0)
        test_list.append(res)
        count += 1
    except:
        continue
        
    if count == 3:
        break

In [ ]:
for t in test_list:
    display(t)

**To format the tables I need to:**
* For each table in the dictionary, set the columns to `'State', 'Senator', 'Party', 'Electoral_History', 'Results', 'Candidates'`.
* If any of the rows look like row 0 & 1, remove them.
* Store the formatted tables back into container

In [ ]:
# test_df['Incumbent'].duplicated(keep=False)
test_df.groupby('Incumbent').indices

* Create a table with names as index and state as values .loc name of candidates to get state value
* Use table of each year's general summary to then pull the general elections out of whole list (index 2 in colleciton) based upon names in table + non-uniform party column
    * Need to merge (if special election) tables in collection index 1

In [ ]:
test = {'1918': ['fire'], '1920': 'jet', '1976': 'hello'}
test['1918'].append('bromine')
for i in test:
    print(i)
print(test['1918'])

* 1924 = 10
* 1926 = 12
* 1928 = 10
* Pull only tables with `shape[1]` of 5 or 6.
* Table 2 is always summary of senate leaders

Why after the 17th amendment?? [Link](https://en.wikipedia.org/wiki/Seventeenth_Amendment_to_the_United_States_Constitution)

For 1920 forward, proceed like 1976. Prior to this, will need to use first table with general info.

* **Clean up NaN value table from top of pages** -- Use index 2 to grab  nested table from `'infobox vevent'`.
* **Take list of states of top tables** and map to dfs
* Get demographic info wikidata
* eda time series of state party (heatmap)
* NLP on names, etc.
* [link for slider viz](https://medium.com/@sjacks/the-journey-to-an-attractive-visualization-bac019506a49)